# Custom Chatbot Project

Datasets used:

- DSQ: Consists of 250 latest Data Science Interview Questions
- 2023-Israel-Hamas-War: Consists of latest on-going war details upto 3rd Dec 2023


We have used DSQ dataset to see how our GPT gets trained and all, it gave us quite a sightings and we came to a conclusion that GPT was already pre-trained.

Now knowing the fact that it was pre-trained and knew all the DSQ data right before we fed it, we decided to go with something latest and on-going event which is "2023-Israel-Hamas-War".

We tested GPT to see if it knew about current on-going events or not and came to know that it knows nothing about current Israel-Hamas on-going war, which makes our dataset perfect for this project.

In [1]:
!pip install cohere openai==0.28 tiktoken numpy scipy pandas

import pandas as pd
import os
import warnings
import numpy as np
import openai
import ast
import tiktoken
from scipy import spatial
from typing import Tuple,List


def warn(*args, **kwargs):
    pass

warnings.warn = warn


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 48.9/48.9 kB 872.4 kB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 76.5/76.5 kB 1.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 MB 15.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.1/3.1 MB 58.3 MB/s eta 0:00:00


In [15]:
from google.colab import drive
drive.mount('/content/gdrive')

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


In [16]:
!ls "/content/gdrive/MyDrive/Colab Notebooks"

 2023-Israel-Hamas-War.csv			 Resume.pdf
 audio_with_hidden_text.wav			 test.wav
'Copy of Untitled3.ipynb'			 Untitled
'Copy of Untitled6.ipynb'			 Untitled0.ipynb
 DSQs_embedded.csv				 Untitled1.ipynb
 DSQs.txt					 Untitled2.ipynb
'Hospitalization Hypothesis Testing (1).ipynb'	 Untitled3.ipynb
'Hospitalization Hypothesis Testing (2).ipynb'	 Untitled4.ipynb
'Hospitalization Hypothesis Testing (3).ipynb'	 Untitled5.ipynb
'Hospitalization Hypothesis Testing.ipynb'	 Untitled6.ipynb
 output10.wav					 video.mp4
 output2.mp4					 wav2lip_gan.pth
 outputs.mp4


## Data Wrangling

TODO: In the cells below, load your chosen dataset into a `pandas` dataframe with a column named `"text"`. This column should contain all of your text data, separated into at least 20 rows.

In [ ]:
file_path = '/content/gdrive/MyDrive/Colab Notebooks/DSQs.txt'
with open(file_path, 'r') as file:
    lines = [line.strip() for line in file]

# Dataframe
df = pd.DataFrame(lines, columns=["text"])

In [ ]:
# Preprocessing

df.to_csv('data/DSQs_preprocessed.csv')

In [ ]:
#Embedding

df = pd.read_csv("/content/gdrive/MyDrive/Colab Notebooks/DSQs_preprocessed.csv")

client = OpenAI(api_key="key") # Place your OpenAI key here

def get_embedding(text, model="text-embedding-ada-002"):
    text = text.replace("\n", " ")
    return client.embeddings.create(input = [text], model=model).data[0].embedding

df['ada_embedding'] = df.text.apply(lambda x: get_embedding(x, model='text-embedding-ada-002'))
df.to_csv('/content/gdrive/MyDrive/Colab Notebooks/DSQs_embedded.csv', index=False)


In [ ]:
df = pd.read_csv("/content/gdrive/MyDrive/Colab Notebooks/DSQs_embedded.csv")
df.head()

,text,ada_embedding
0,How do you ensure that your data science model...,"[-0.0017857637722045183, -0.010468040592968464..."
1,What tools and techniques do you use for data ...,"[0.0013928849948570132, -0.0002107492036884650..."
2,Can you explain the concept and importance of ...,"[0.015783732756972313, -0.008996326476335526, ..."
3,How would you handle imbalanced datasets in a ...,"[-0.0062577431090176105, 0.012802968733012676,..."
4,What is the difference between a Type I and Ty...,"[0.01271655410528183, -0.0003393392253201455, ..."


In [ ]:
df.rename(columns={'ada_embedding': 'embedding'}, inplace=True)
df.head()

,text,embedding
0,How do you ensure that your data science model...,"[-0.0017857637722045183, -0.010468040592968464..."
1,What tools and techniques do you use for data ...,"[0.0013928849948570132, -0.0002107492036884650..."
2,Can you explain the concept and importance of ...,"[0.015783732756972313, -0.008996326476335526, ..."
3,How would you handle imbalanced datasets in a ...,"[-0.0062577431090176105, 0.012802968733012676,..."
4,What is the difference between a Type I and Ty...,"[0.01271655410528183, -0.0003393392253201455, ..."


## Custom Query Completion

TODO: In the cells below, compose a custom query using your chosen dataset and retrieve results from an OpenAI `Completion` model.

In [5]:
openai.api_key = 'key'  # Replace with your actual API key

In [ ]:
def strings_ranked_by_relatedness(
        query: str,
        df: pd.DataFrame,
        relatedness_fn=lambda x, y: 1 - spatial.distance.cosine(x, y),
        top_n: int = 100
) -> Tuple[List[str], List[float]]:
    """Returns a list of strings and relatednesses, sorted from most related to least."""
    query_embedding_response = openai.Embedding.create(
        model="text-embedding-ada-002",
        input=query,
    )
    query_embedding = np.array(query_embedding_response["data"][0]["embedding"])
    if len(query_embedding.shape) != 1:
        query_embedding = np.reshape(query_embedding, -1)  # Reshape to 1-D if not already

    strings_and_relatednesses = []
    for _, row in df.iterrows():
        # Convert the string representation of embedding to numpy array
        row_embedding = np.fromstring(row["embedding"].strip('[]'), sep=',')
        relatedness = relatedness_fn(query_embedding, row_embedding)
        strings_and_relatednesses.append((row["text"], relatedness))

    strings_and_relatednesses.sort(key=lambda x: x[1], reverse=True)
    strings, relatednesses = zip(*strings_and_relatednesses)
    return list(strings)[:top_n], list(relatednesses)[:top_n]

In [ ]:
# examples
strings, relatednesses = strings_ranked_by_relatedness("Machine learning", df, top_n=5)
for string, relatedness in zip(strings, relatednesses):
    print(f"{relatedness=:.3f}")
    display(string)

relatedness=0.855


'Can you discuss your experience with automated machine learning (AutoML)?'

relatedness=0.852


'Can you explain the use of optimization algorithms in machine learning?'

relatedness=0.852


'Can you explain the application of machine learning in weather forecasting and climate modeling?'

relatedness=0.852


'Can you discuss the application of quantum machine learning?'

relatedness=0.851


'Can you explain the role of machine learning in audio signal processing and music analysis?'

Relatedness shows 85% accuracy which is great in my opinion as our dataset is completely based on questions and not on text.

## Custom Performance Demonstration

TODO: In the cells below, demonstrate the performance of your custom query using at least 2 questions. For each question, show the answer from a basic `Completion` model query as well as the answer from your custom query.

In [ ]:
# Number of tokens used

def num_tokens(text: str, model: str = "gpt-3.5-turbo") -> int:
    """Return the number of tokens in a string."""
    encoding = tiktoken.encoding_for_model(model)
    return len(encoding.encode(text))


In [ ]:
# Setting Personality

def query_message(
    query: str,
    df: pd.DataFrame,
    model: str,
    token_budget: int
) -> str:
    """Return a message for GPT, with relevant source texts pulled from a dataframe."""
    strings, relatednesses = strings_ranked_by_relatedness(query, df)
    introduction = """
Task: Assume the role of a Senior Data Scientist at Google, specializing in recent technologies like LLMs and long-chain algorithms.

1. Interview Simulation: Conduct mock interviews for positions like Data Scientist, Data Analyst, AI Engineer, and ML Engineer. Pose relevant, up-to-date data science questions.
2. Response Evaluation: After receiving an answer, rate it on a scale of 1-10 based on accuracy and relevance.
3. Feedback Provision: Offer concise, specific feedback, highlighting inaccuracies or areas for improvement in the answer.
4. Topic Adherence: Ensure the discussion remains focused on data science. If the conversation strays, remind participants to stay on topic.
5. Expertise Utilization: Utilize your expertise to provide responses that reflect current industry standards and knowledge in data science.

The goal is to simulate a realistic interview environment, providing practical learning opportunities in data science.
Use the questions below for reference on what to ask."""
    question = f"\n\nQuestion: {query}"
    message = introduction
    for string in strings:
        if (
            num_tokens(message + question, model=model)
            > token_budget
        ):
            break
        else:
            message += ""
    return message + question


# Asking questions

def ask(
    query: str,
    df: pd.DataFrame = df,
    model: str = "gpt-3.5-turbo",
    token_budget: int = 4096 - 500,
    print_message: bool = False,
) -> str:
    """Answers a query using GPT and a dataframe of relevant texts and embeddings."""
    message = query_message(query, df, model=model, token_budget=token_budget)
    if print_message:
        print(message)
    messages = [
        {"role": "system", "content": "Here are the questions: "},
        {"role": "user", "content": message},
    ]
    response = openai.ChatCompletion.create(
        model=model,
        messages=messages,
        temperature=0
    )
    response_message = response["choices"][0]["message"]["content"]
    return response_message

In [ ]:
ask('Give me one question on machine learning.')

"Sure! Here's a question on machine learning:\n\nQuestion: Can you explain the concept of overfitting in machine learning and how it can be addressed?\n\nPlease provide your answer, and I will evaluate it on a scale of 1-10 based on accuracy and relevance."

In [ ]:
ask("Who won valorant esports in 2022?")

"I'm sorry, but I'm unable to answer that question as it falls outside the scope of my expertise as a language model AI. My knowledge is based on a wide range of topics, but it does not include up-to-date information on specific esports events or winners. Is there anything else I can assist you with related to data science or technology?"

In [ ]:
ask('Give me a question based on the given dataset')

"Sure! Here's a question based on a dataset:\n\nQuestion: You are given a dataset containing information about customer transactions at an online retail store. The dataset includes columns such as customer ID, product ID, purchase date, quantity, and price. Your task is to analyze the data and provide insights to improve the store's sales performance. What steps would you take to approach this analysis?\n\nPlease provide your answer, and I will evaluate it based on accuracy and relevance."

In [ ]:
ask('Machine learning')

'1. What is the difference between supervised and unsupervised learning?\n2. Can you explain the bias-variance tradeoff in machine learning?\n3. What is regularization in machine learning and why is it important?\n4. How do decision trees work in machine learning?\n5. What is the purpose of cross-validation in machine learning?\n6. Can you explain the concept of feature selection in machine learning?\n7. What is the difference between bagging and boosting in ensemble learning?\n8. Can you explain the concept of gradient descent in machine learning?\n9. What is the difference between classification and regression in machine learning?\n10. Can you explain the concept of overfitting in machine learning and how to address it?\n\nRemember to rate the responses on a scale of 1-10 based on accuracy and relevance, and provide specific feedback to help the interviewee improve their answers.'

In [ ]:
ask('mathematical modeling')

'1. Can you explain what mathematical modeling is and its importance in data science?\n\n2. How would you approach building a mathematical model for a real-world problem? Can you walk me through the steps involved?\n\n3. What are some common mathematical models used in data science, and what are their applications?\n\n4. Can you explain the concept of parameter estimation in mathematical modeling? How is it done, and why is it important?\n\n5. How do you validate a mathematical model? What techniques or methods do you use to ensure its accuracy and reliability?\n\n6. Can you discuss the limitations or challenges of mathematical modeling in data science? How do you address these challenges?\n\n7. In your experience, what are some best practices for effectively communicating the results of a mathematical model to stakeholders or non-technical audiences?\n\n8. How do you handle situations where the data available for modeling is incomplete or contains outliers? What techniques or approach

Ok so it successfully generated us questions and some of them weren't on our dataset.

Let's try changing the prompt and see what output it gives.



In [ ]:
# Setting Personality

def query_message(
    query: str,
    df: pd.DataFrame,
    model: str,
    token_budget: int
) -> str:
    """Return a message for GPT, with relevant source texts pulled from a dataframe."""
    strings, relatednesses = strings_ranked_by_relatedness(query, df)
    introduction = introduction = 'Use the below questions to generate questions. If you arent able to generate then simply give "Sorry i cant generate it." as response'
    question = f"\n\nQuestion: {query}"
    message = introduction
    for string in strings:
        if (
            num_tokens(message + question, model=model)
            > token_budget
        ):
            break
        else:
            message += ""
    return message + question


# Asking questions

def ask(
    query: str,
    df: pd.DataFrame = df,
    model: str = "gpt-3.5-turbo",
    token_budget: int = 4096 - 500,
    print_message: bool = False,
) -> str:
    """Answers a query using GPT and a dataframe of relevant texts and embeddings."""
    message = query_message(query, df, model=model, token_budget=token_budget)
    if print_message:
        print(message)
    messages = [
        {"role": "system", "content": "Here are the questions: "},
        {"role": "user", "content": message},
    ]
    response = openai.ChatCompletion.create(
        model=model,
        messages=messages,
        temperature=0
    )
    response_message = response["choices"][0]["message"]["content"]
    return response_message

In [ ]:
ask("Give me one question on Machine learning.")

'What are some common algorithms used in machine learning?'

Again, it generated new question which wasn't in our dataset.

In [ ]:
ask("Whats after this question: How do you use convolutional neural networks for object detection?")

"Sorry, I can't generate it."

Lets make it stick to generating our questions strictly.

In [ ]:
file_path = '/content/gdrive/MyDrive/Colab Notebooks/DSQs.txt'
with open(file_path, 'r') as file:
    lines = [line.strip() for line in file]

In [ ]:
questions_string = '\n'.join(lines)
query = f"""Use the below questions, remember them and strictly stick to generating one of those questions. As an example, if a user says: 'Machine learning', you should search for the first machine learning question that pops up in these lines and give the output.\n\nQuestions:\n{questions_string}"""
# Prepare the messages for the ChatCompletion request
messages = [
    {'role': 'system', 'content': query},
    {'role': 'user', 'content': 'Give me two questions on machine learning.'},
]

# Create a response using OpenAI's ChatCompletion
response = openai.ChatCompletion.create(
    messages=messages,
    model='gpt-3.5-turbo',
    temperature=0,
)

print(response['choices'][0]['message']['content'])

1. How do you ensure that your data science models are fair and unbiased?
2. How do you approach feature selection in a dataset?


We got an better understanding on how to train and test GPT on our dataset, but what if GPT was already pre-trained before our training, like it knew and was familiar with our dataset ?

# Assuming GPT-3.5 is trained on our dataset before implementation, let's feed something new and latest data for better understanding on how it works.

In [10]:
query = f"""Answer the question"""
# Prepare the messages for the ChatCompletion request
messages = [
    {'role': 'system', 'content': query},
    {'role': 'user', 'content': 'When did Hamas-Israel war started ?'},
]

# Create a response using OpenAI's ChatCompletion
response = openai.ChatCompletion.create(
    messages=messages,
    model='gpt-3.5-turbo',
    temperature=0,
)

print(response['choices'][0]['message']['content'])

The most recent major conflict between Hamas and Israel, known as the 2021 Israel-Hamas conflict, began on May 10, 2021.


In [11]:
query = f"""Answer the question"""
# Prepare the messages for the ChatCompletion request
messages = [
    {'role': 'system', 'content': query},
    {'role': 'user', 'content': 'Do you know about current 2023 on-going war on Israel-Hamas ?'},
]

# Create a response using OpenAI's ChatCompletion
response = openai.ChatCompletion.create(
    messages=messages,
    model='gpt-3.5-turbo',
    temperature=0,
)

print(response['choices'][0]['message']['content'])

Yes, I am aware of the ongoing conflict between Israel and Hamas in 2023. The conflict began in May 2023 when tensions escalated between Israel and Hamas, a Palestinian militant group that controls the Gaza Strip. The immediate trigger for the conflict was a series of clashes between Palestinians and Israeli security forces in Jerusalem, particularly around the Al-Aqsa Mosque compound, which is a holy site for both Muslims and Jews.

Following these clashes, Hamas launched rocket attacks towards Israeli cities, and Israel responded with airstrikes targeting Hamas infrastructure in the Gaza Strip. The conflict has resulted in significant casualties and destruction on both sides, with civilians bearing the brunt of the violence.

Efforts to broker a ceasefire and de-escalate the situation have been ongoing, with various international actors, including the United Nations, Egypt, and Qatar, involved in mediation efforts. However, as of now, a lasting ceasefire has not been achieved, and th

In [12]:
query = f"""Answer the question"""
# Prepare the messages for the ChatCompletion request
messages = [
    {'role': 'system', 'content': query},
    {'role': 'user', 'content': 'Do you Know what happened in November 2023 in Israel-Hamas war?'},
]

# Create a response using OpenAI's ChatCompletion
response = openai.ChatCompletion.create(
    messages=messages,
    model='gpt-3.5-turbo',
    temperature=0,
)

print(response['choices'][0]['message']['content'])

I'm sorry, but as an AI language model, I don't have access to real-time information or the ability to browse the internet. Therefore, I cannot provide specific details about events that may occur in the future, such as the Israel-Hamas war in November 2023. It's best to refer to reliable news sources for the most up-to-date information on current events.


We can clearly see that it doesn't know anything about current on-going 2023 war on Israel-Hamas, perfect opportunity for us to train this GPT on our dataset

### Now lets make embeddings

In [17]:
# Load data
df = pd.read_csv("/content/gdrive/MyDrive/Colab Notebooks/2023-Israel-Hamas-War.csv")

# Ensure all text entries are strings
df['text'] = df['text'].astype(str)

# OpenAI client
openai.api_key = "key"

# Function to get embedding
def get_embedding(text, model="text-embedding-ada-002"):
    text = text.replace("\n", " ")
    return openai.Embedding.create(input=[text], model=model).data[0].embedding

df['embedding'] = df['text'].apply(lambda x: get_embedding(x, model='text-embedding-ada-002'))

# Save to CSV
df.to_csv("/content/gdrive/MyDrive/Colab Notebooks/War_embedded.csv")

In [20]:
df = pd.read_csv("/content/gdrive/MyDrive/Colab Notebooks/War_embedded.csv")
df.head()

,Unnamed: 0,text,embedding
0,0,2023 Israel Hamas war An ongoing armed confli...,"[-0.03198346123099327, -0.020022273063659668, ..."
1,1,"On that day, Hamas-led Palestinian militants[o...","[-0.0326392836868763, -0.007378561422228813, 0..."
2,2,The surprise attack comprised a barrage of roc...,"[-0.011929569765925407, -0.008353897370398045,..."
3,3,At least 846 Israeli civilians and 324 soldier...,"[-0.004879564046859741, 0.004347653593868017, ..."
4,4,Hamas stated that its attack was in response t...,"[-0.03601551800966263, -0.014106078073382378, ..."


In [ ]:
df.drop('Unnamed: 0', axis=1, inplace=True)

In [25]:
df.to_csv("/content/gdrive/MyDrive/Colab Notebooks/War_embedded.csv")

In [26]:
df.head()

,text,embedding
0,2023 Israel Hamas war An ongoing armed confli...,"[-0.03198346123099327, -0.020022273063659668, ..."
1,"On that day, Hamas-led Palestinian militants[o...","[-0.0326392836868763, -0.007378561422228813, 0..."
2,The surprise attack comprised a barrage of roc...,"[-0.011929569765925407, -0.008353897370398045,..."
3,At least 846 Israeli civilians and 324 soldier...,"[-0.004879564046859741, 0.004347653593868017, ..."
4,Hamas stated that its attack was in response t...,"[-0.03601551800966263, -0.014106078073382378, ..."


In [27]:
def strings_ranked_by_relatedness(
        query: str,
        df: pd.DataFrame,
        relatedness_fn=lambda x, y: 1 - spatial.distance.cosine(x, y),
        top_n: int = 100
) -> Tuple[List[str], List[float]]:
    """Returns a list of strings and relatednesses, sorted from most related to least."""
    query_embedding_response = openai.Embedding.create(
        model="text-embedding-ada-002",
        input=query,
    )
    query_embedding = np.array(query_embedding_response["data"][0]["embedding"])
    if len(query_embedding.shape) != 1:
        query_embedding = np.reshape(query_embedding, -1)  # Reshape to 1-D if not already

    strings_and_relatednesses = []
    for _, row in df.iterrows():
        # Convert the string representation of embedding to numpy array
        row_embedding = np.fromstring(row["embedding"].strip('[]'), sep=',')
        relatedness = relatedness_fn(query_embedding, row_embedding)
        strings_and_relatednesses.append((row["text"], relatedness))

    strings_and_relatednesses.sort(key=lambda x: x[1], reverse=True)
    strings, relatednesses = zip(*strings_and_relatednesses)
    return list(strings)[:top_n], list(relatednesses)[:top_n]

In [28]:
# examples
strings, relatednesses = strings_ranked_by_relatedness("December", df, top_n=5)
for string, relatedness in zip(strings, relatednesses):
    print(f"{relatedness=:.3f}")
    display(string)

relatedness=0.819


'On 1 December, combat resumed following the expiration of the truce'

relatedness=0.811


nan

relatedness=0.796


'Resumption of hostilities (1 December present)  The ceasefire ended in the morning and both parties confirmed that military operations resumed'

relatedness=0.790


'28 November  Both Israel and Hamas accused each other of violating the truce'

relatedness=0.788


'1 December  Leaflets were dropped with maps into Khan Yunis depicting hundreds of evacuation zones'

### We are good to proceed with our QA

In [38]:
# Number of tokens used

def num_tokens(text: str, model: str = "gpt-3.5-turbo") -> int:
    """Return the number of tokens in a string."""
    encoding = tiktoken.encoding_for_model(model)
    return len(encoding.encode(text))
# Setting Personality

def query_message(query: str, df: pd.DataFrame, model: str, token_budget: int) -> str:
    strings, relatednesses = strings_ranked_by_relatedness(query, df)
    introduction = "Answer the question based on the data that was provided to you:"
    question = f"\n\nQuestion: {query}"
    message = introduction
    for string in strings:
        potential_message = message + "\n\n" + string + question
        if num_tokens(potential_message, model=model) > token_budget:
            break
        else:
            message = potential_message
    return message

# Revised ask function
def ask(query: str, df: pd.DataFrame, model: str = "gpt-3.5-turbo", token_budget: int = 4096 - 500, print_message: bool = False) -> str:
    message = query_message(query, df, model=model, token_budget=token_budget)
    if print_message:
        print(message)
    messages = [
        {"role": "system", "content": "Answer this question based on the following context."},
        {"role": "user", "content": message},
    ]
    response = openai.ChatCompletion.create(
        model=model,
        messages=messages,
        temperature=0
    )
    response_message = response.choices[0].message.content
    return response_message

In [56]:
ask("Events happened on december of war?",df)

"Based on the provided data, the following events happened in December of the war:\n\n- On 1 December, combat resumed following the expiration of the truce.\n- Leaflets were dropped with maps into Khan Yunis depicting hundreds of evacuation zones on 1 December.\n- Multiple conflicts escalated into outright wars.\n- On 28 November, both Israel and Hamas accused each other of violating the truce.\n- Israel's declaration of a state of war marked the start of the most significant military escalation in the region since the Yom Kippur War in 1973."

In [43]:
ask("On 1 December, combat resumed following the expiration of the truce ?",df)

'Yes, on 1 December, combat resumed following the expiration of the truce.'

# Now lets implement the while loop

In [52]:
def main():
    # Load your DataFrame
    df = pd.read_csv("War_embedded.csv")

    # Looping
    while True:
        user_input = input("Ask a question (or type 'exit' to stop): ")
        if user_input.lower() == 'exit':
            break

        response = ask(user_input, df)
        print("Response:", response)

if __name__ == "__main__":
    main()


Ask a question (or type 'exit' to stop): Can you tell me when this israel-hamas war started ?
Response: Based on the information provided, the Israel-Hamas war started on 7 October 2023.
Ask a question (or type 'exit' to stop): Can you tell me how many people died during this war ?
Response: Based on the information provided, it is not possible to determine the exact number of people who died during this war. The data includes various incidents and conflicts, but does not provide a comprehensive count of the total casualties.
Ask a question (or type 'exit' to stop): What events happened on november of war ?
Response: Based on the data provided, the events that happened in November of the war include:

- On 1 November, the first group of evacuees left Gaza into Egypt.
- On 28 November, both Israel and Hamas accused each other of violating the truce.
- On 21 November, the IDF moved its frontline to encircle the Jabaliya camp where they are battling Palestinian militants and attempting to